In [1]:
import os
import pickle
import pandas as pd
pd.set_option('display.max_rows', None)
import matplotlib.pyplot as plt
from sklearn import svm
import matplotlib.pyplot as plt
from Haldane_anis_supervise_class import *
from partial import *

In [ ]:
L = 31  # number of particles must be odd
bond = 50  # reasonable amount
p = partial(L, bond)

In [4]:
train_set,_ = p.generate_dataset(generate_test_set=False)

Starting dataset generation — this step includes both the training and test sets. Please be patient...


Generating train set: 100%|██████████| 187/187 [00:04<00:00, 40.07it/s] 


Train_set is generated in 5.08 seconds.


In [ ]:
keep = [0,1] # keep the part that we wanna do partial trace over that
partial_density_matrix_train, partial_density_matrix_test = p.partial_density_matrix(keep=keep)
kernel_partial_train = p.gram_train_partial(partial_train=partial_density_matrix_train, keep=keep)
kernel_partial_test = p.gram_test_partial(partial_train=partial_density_matrix_train, partial_test=partial_density_matrix_test, keep=keep)

In [ ]:
keep = [0,1,2] # keep the part that we wanna do partial trace over that
partial_density_matrix_train, partial_density_matrix_test = p.partial_density_matrix(keep=keep)
kernel_partial_train = p.gram_train_partial(partial_train=partial_density_matrix_train, keep=keep)
kernel_partial_test = p.gram_test_partial(partial_train=partial_density_matrix_train, partial_test=partial_density_matrix_test, keep=keep)

In [ ]:
keep = [0,1,2,3] # keep the part that we wanna do partial trace over that
partial_density_matrix_train, partial_density_matrix_test = p.partial_density_matrix(keep=keep)
kernel_partial_train = p.gram_train_partial(partial_train=partial_density_matrix_train, keep=keep)
kernel_partial_test = p.gram_test_partial(partial_train=partial_density_matrix_train, partial_test=partial_density_matrix_test, keep=keep)

In [ ]:
keep = [24,25,26] # keep the part that we wanna do partial trace over that
partial_density_matrix_train, partial_density_matrix_test = p.partial_density_matrix(keep=keep)
kernel_partial_train = p.gram_train_partial(partial_train=partial_density_matrix_train, keep=keep)
kernel_partial_test = p.gram_test_partial(partial_train=partial_density_matrix_train, partial_test=partial_density_matrix_test, keep=keep)

In [ ]:
keep = [24,25,26,27] # keep the part that we wanna do partial trace over that
partial_density_matrix_train, partial_density_matrix_test = p.partial_density_matrix(keep=keep)
kernel_partial_train = p.gram_train_partial(partial_train=partial_density_matrix_train, keep=keep)
kernel_partial_test = p.gram_test_partial(partial_train=partial_density_matrix_train, partial_test=partial_density_matrix_test, keep=keep)